# Databricks - Credit Scoring

## Introduction

Banks play a crucial role in market economies. They decide who can get finance and on what terms and can make or break investment decisions. For markets and society to function, individuals and companies need access to credit. 

Credit scoring algorithms, which make a guess at the probability of default, are the method banks use to determine whether or not a loan should be granted. 

## The problem

Down below you will find a possible solution to the challenge described in [c/GiveMeSomeCredit](https://www.kaggle.com/c/GiveMeSomeCredit) where participants where required to improve on the state of the art in credit scoring, by predicting the probability that somebody will experience financial distress in the next two years. 

## The data

The training data contains the following variables:


| **Variable   Name**                  | **Description**                                                                                                                                              | **Type**   |
|--------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------|------------|
| SeriousDlqin2yrs                     | Person   experienced 90 days past due delinquency or worse                                                                                                   | *Y/N*      |
| RevolvingUtilizationOfUnsecuredLines | Total   balance on credit cards and personal lines of credit except real estate and   no installment debt like car loans divided by the sum of credit limits | percentage |
| age                                  | Age of borrower in   years                                                                                                                                   | integer    |
| NumberOfTime30-59DaysPastDueNotWorse | Number of times   borrower has been 30-59 days past due but no worse in the last 2 years.                                                                    | integer    |
| DebtRatio                            | Monthly debt   payments, alimony,living costs divided by monthy gross income                                                                                 | percentage |
| MonthlyIncome                        | Monthly income                                                                                                                                               | real       |
| NumberOfOpenCreditLinesAndLoans      | Number of Open loans   (installment like car loan or mortgage) and Lines of credit (e.g. credit   cards)                                                     | integer    |
| NumberOfTimes90DaysLate              | Number of times   borrower has been 90 days or more past due.                                                                                                | integer    |
| NumberRealEstateLoansOrLines         | Number of mortgage   and real estate loans including home equity lines of credit                                                                             | integer    |
| NumberOfTime60-89DaysPastDueNotWorse | Number of times   borrower has been 60-89 days past due but no worse in the last 2 years.                                                                    | integer    |
| NumberOfDependents                   | Number of dependents   in family excluding themselves (spouse, children etc.)                                                                                | integer    |

The **SeriousDlqin2yrs** is the dependent variable of the dataset, or better named the **label**. This is a boolean value which details if a certain individual has experienced a deliquency of 90 days past due or worse in the last 2 years.

You can get the training data from [here](https://github.com/dlawrences/GlobalAINightBucharest/blob/master/data/cs-training.csv).

This dataset should be used for:
- creating two smaller sets, one for the actual training (e.g. 80%) and one for testing (e.g. 20%)
- during cross validation, if you want to do the validation on multiple different folds of data to manage better the bias and the variance

The benchmark/real unseen data you could use to test your model predictions may be downloaded from [here](https://github.com/dlawrences/GlobalAINightBucharest/blob/master/data/cs-test.csv).

## The Data Science Process

This is the outline of the process we'll be following in this workshop.

![Data Science Process](https://raw.githubusercontent.com/neaorin/PredictTheWorldCup/master/images/datascience_process.jpg)

## Data import

Before starting to do anything else, we need to import the data. First, let's download both datasets and store them in DBFS.

In [3]:
import urllib.request

training_data_url = "https://raw.githubusercontent.com/dlawrences/GlobalAINightBucharest/master/data/cs-training.csv"
training_data_filename = "cs_training.csv"

test_data_url = "https://raw.githubusercontent.com/dlawrences/GlobalAINightBucharest/master/data/cs-test.csv"
test_data_filename = "cs_test.csv"

dbfs_data_folder = "dbfs/FileStore/data/"
project_name = 'credit-scoring'

dbfs_project_folder = dbfs_data_folder + project_name + "/"

# Download files and move them to the final directory in DBFS
urllib.request.urlretrieve(training_data_url, "/tmp/" + training_data_filename)
urllib.request.urlretrieve(test_data_url, "/tmp/" + test_data_filename)

# Create the project directory if it does not exist and move files to it
dbutils.fs.mkdirs(dbfs_project_folder)
dbutils.fs.mv("file:/tmp/" + training_data_filename, dbfs_project_folder)
dbutils.fs.mv("file:/tmp/" + test_data_filename, dbfs_project_folder)

# List the contents of the directory
dbutils.fs.ls(dbfs_project_folder)

Out[1]: [FileInfo(path='dbfs:/dbfs/FileStore/data/credit-scoring/cs_test.csv', name='cs_test.csv', size=4937873),
 FileInfo(path='dbfs:/dbfs/FileStore/data/credit-scoring/cs_training.csv', name='cs_training.csv', size=7347657)]

In [4]:
import numpy as np # library for linear algebra and stuff
import pandas as pd # library for data processing, I/O on csvs etc
import matplotlib.pyplot as plt # library for plotting
import seaborn as sns # a library which is better for plotting

# File location and type
file_location = dbfs_project_folder + training_data_filename
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
creditSDF = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(creditSDF)

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766126609,45,2,0.802982129,9120,13,0,6,0,2
2,0,0.957151019,40,0,0.121876201,2600,4,0,0,0,1
3,0,0.65818014,38,1,0.085113375,3042,2,1,0,0,0
4,0,0.233809776,30,0,0.036049682,3300,5,0,0,0,0
5,0,0.9072394,49,1,0.024925695,63588,7,0,1,0,0
6,0,0.213178682,74,0,0.375606969,3500,3,0,1,0,1
7,0,0.305682465,57,0,5710.0,null,8,0,3,0,0
8,0,0.754463648,39,0,0.209940017,3500,8,0,0,0,0
9,0,0.116950644,27,0,46.0,null,2,0,0,0,null
10,0,0.189169052,57,0,0.606290901,23684,9,0,4,0,2


Now that we've loaded the dataset, the first thing we're going to do is set aside a part of it - 25 to 30 percent is a usual percentage - and not touch it until it's time to test our models.

In [6]:
temp_table_name = "trainingSDF"

# Split the data into training and test sets (25% held out for testing)
(trainingSDF, testingSDF) = creditSDF.randomSplit([0.75, 0.25], seed=1)

# Make the dataframe available in the SQL context
trainingSDF.createOrReplaceTempView(temp_table_name)

In [7]:
# Sample out 10 rows of the dataset
display(trainingSDF.sample(False, 0.1, seed=0).limit(10))

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
27,0,0.052436094,58,0,0.097672186,8333,22,0,1,0,0
46,0,6.02723E-4,70,0,0.177787036,16800,12,0,1,0,2
54,0,0.028562367,51,0,0.306381499,5750,5,0,1,0,0
60,0,0.655569945,35,0,0.261609907,5813,10,0,0,0,2
64,0,0.009788862,46,0,1.051397656,3326,6,0,2,0,2
72,0,0.142013011,67,0,1824.0,null,7,0,2,0,0
103,0,0.634485057,52,0,0.138238573,2690,2,0,0,0,0
110,0,0.041257909,61,0,4739.0,null,11,0,4,0,null
123,0,0.297594642,67,0,0.239507959,4145,13,0,0,0,0
139,0,0.093962275,55,0,0.112051635,9450,6,0,2,0,0


In [8]:
# Inspect the schema
trainingSDF.printSchema()

root
-- Idx: integer (nullable = true)
-- SeriousDlqin2yrs: integer (nullable = true)
-- RevolvingUtilizationOfUnsecuredLines: double (nullable = true)
-- age: integer (nullable = true)
-- NumberOfTime30-59DaysPastDueNotWorse: integer (nullable = true)
-- DebtRatio: double (nullable = true)
-- MonthlyIncome: integer (nullable = true)
-- NumberOfOpenCreditLinesAndLoans: integer (nullable = true)
-- NumberOfTimes90DaysLate: integer (nullable = true)
-- NumberRealEstateLoansOrLines: integer (nullable = true)
-- NumberOfTime60-89DaysPastDueNotWorse: integer (nullable = true)
-- NumberOfDependents: integer (nullable = true)

In [9]:
# Check of the summary statistics of the features
display(trainingSDF.describe())

summary,Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,112610,112610,112610,112610,112610,112610,90323,112610,112610,112610,112610,109673
mean,74982.02763520114,0.06655714412574372,6.029122396750183,52.330832075304144,0.4257348370482195,352.5467977490791,6647.620417833774,8.450031080721073,0.27232039783322975,1.0170411153538763,0.24635467542846995,0.7544518705606668
stddev,43267.1824110714,0.2492545734778176,253.13410947895514,14.792492602445721,4.263688445126266,2230.784927828637,12296.248257382455,5.1483988188414065,4.240935535626126,1.1218679345702816,4.226870854098003,1.1089255063495669
min,1,0,0.0,0,0,0.0,0,0,0,0,0,0
max,149999,1,50708.0,109,98,329664.0,1794060,57,98,29,98,10


In [10]:
# highlight how many missing values we have for every feature
from pyspark.sql.functions import lit, col

rows = trainingSDF.count()
summary = trainingSDF.describe().filter(col("summary") == "count")
display(summary.select(*((lit(rows)-col(c)).alias(c) for c in trainingSDF.columns)))

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0.0,0.0,0.0,0.0,0.0,0.0,22287.0,0.0,0.0,0.0,0.0,2937.0


Quick conclusions:
- there are a lot of null values for **MonthlyIncome** and **NumberOfDependents**; we will analyse how to impute these next
- the minimum value for the **age** variable is 0 and it presents an outlier/bad data; this will be imputed with the median
- the maximum value of **329664** for the **DebtRatio** variable is rather weird given this variable is a mere percentage; from a modelling perspective, thus we will need to understand why there are such big values and decide what to do with them
- the maximum value of **50708** for the **RevolvingUtilizationOfUnsecuredLines** variable is rather weird given this variable is a mere percentage; rom a modelling perspective, thus we will need to understand why there are such big values and decide what to do with them

## Exploratory Data Analysis & Data Cleaning

We are going to take step by step most of the interesting columns that need visualizing and cleansing to be done.

### Target class - SeriousDlqin2yrs

Let's understand the distribution of our target class (**SeriousDlqin2yrs**). This could very well influence the algorithm we will want to use to model the problem.

In [13]:
%sql

select SeriousDlqin2yrs, count(*) as TotalCount from trainingSDF group by SeriousDlqin2yrs

SeriousDlqin2yrs,TotalCount
1,7495
0,105115


There seems to be a lot of **class imbalance** going on. Let's understand the positive event rate in our target class.

In [15]:
class_0 = trainingSDF.filter(trainingSDF.SeriousDlqin2yrs == 0).count()
class_1 = trainingSDF.filter(trainingSDF.SeriousDlqin2yrs == 1).count()

print("Total number of observations with a class of 0: {}".format(class_0))
print("Total number of observations with a class of 1: {}".format(class_1))
print("Positive event rate: {} %".format(class_1/(class_0+class_1) * 100))

Total number of observations with a class of 0: 105115
Total number of observations with a class of 1: 7495
Positive event rate: 6.655714412574372 %

A positive event rate of 6.6% is by no means ideal. Going through with this distribution for the target class may mean that the minorit class will be ignored by the algorithm we are going to use to model the problem, thus the model will be biased to customers which are not likely to default.

A couple of ideas which we are going to take into consideration going further to go around this problem:
- given we have a lot of training data (100k+ observations), we may actually considering resampling the dataset.
- we are going to use an evaluation metric which compensates the imbalance between classes, e.g. **ROC AUC**

### Age variable

We are interested in knowing the distribution of the **age** variable. 

We are not looking for customers under the legal age of 18 years. If any, we will impute the age of these with the median of the column.

In [18]:
import matplotlib.ticker as ticker

# spark.sql does not have any histogram method, however the RDD api does
age_histogram = trainingSDF.select('age').rdd.flatMap(lambda x: x).histogram(10)

fig, ax = plt.subplots()

# the computed histogram needs to be loaded in a pandas dataframe so we will be able to plot it using sns
age_histogram_df = pd.DataFrame(
    list(zip(*age_histogram)), 
    columns=['bin', 'frequency']
)

ax = sns.barplot(x = "bin", y = "frequency", data = age_histogram_df)

ax.get_xaxis().set_major_formatter(ticker.FuncFormatter(lambda x, p: format(age_histogram_df.iloc[x]['bin'], '.1f')))

display(fig)

It seems there may be customers under the legal age. Let's see how many.

In [20]:
# We can use the filter method to understand what are the observations for which the customers falls under the legal age.
display(trainingSDF.filter(trainingSDF.age < 18))

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
65696,0,0.9999999,0,1,0.436927179,6000,6,0,2,0,2


Fortunately there is only one. Let's impute this value with the median.

In [22]:
# Import functions which will help us code an if statement
from pyspark.sql import functions as F

def imputeAgeWithMedian(df, medianAge):

  # Update with the median for the rows where the age columnis equal to 0
  df = df.withColumn('age',
                                       F.when(
                                           F.col('age') == 0,
                                           medianAge
                                       ).otherwise(
                                           F.col('age')
                                       )
                )
  
  return df

# Compute the median of the age variable
trainingMedianAge = np.median(trainingSDF.select('age').dropna().collect())
trainingSDF = imputeAgeWithMedian(trainingSDF, trainingMedianAge)

# Check to see that the only row shown above has a new age value
display(trainingSDF.filter(trainingSDF.Idx == 65696))

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
65696,0,0.9999999,52.0,1,0.436927179,6000,6,0,2,0,2


Finally, let's check the distribution of the age for each group, based on the values for the **SeriousDlqin2yrs** target variable.

We're going to use a [box and whiskers plot](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51?gi=9e6b6042f263) to better visualize the distribution.

In [24]:
fig, ax = plt.subplots()

ax = sns.boxplot(x="SeriousDlqin2yrs", y="age", data = trainingSDF.toPandas())

display(fig)

In [25]:
%sql

SELECT SeriousDlqin2yrs, age FROM trainingSDF

SeriousDlqin2yrs,age
1,45
0,40
0,38
0,30
0,74
0,57
0,39
0,27
0,57
0,51


Based on the cleaned age column, let's create an age banding column (bins) which might be better predictors to credit risk.

For this example, we are going to use the bins included in this paper: [figure in paper](https://www.researchgate.net/figure/Percentage-of-default-risk-among-different-age-groups_fig2_268345909).

> NOTE: For simplicity we are using a [Spark UDF](https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html) (User-defined function), although that may pose performance problems if the dataset is large. Consider using Scala for the production data preparation pipeline once the data scientist has defined and tested one that should be used in production.

In [27]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def bandingFunction(age):
  if (age < 25):
    return '18-25'
  elif (age >= 25 and age < 30): 
    return '25-29'
  elif (age >= 30 and age < 35):
    return '30-34'
  elif (age >= 35 and age < 40):
    return '35-39'
  elif (age >= 40 and age < 45):
    return '40-44'
  elif (age >= 45 and age < 50):
    return '45-49'
  elif (age >= 50 and age < 55):
    return '50-54'
  elif (age >= 55 and age < 60):
    return '55-59'
  elif (age >= 60 and age < 65):
    return '60-64'
  elif (age >= 65 and age < 70):
    return '65-69'
  elif (age >= 70 and age < 75):
    return '70-74'
  elif (age >= 75): 
    return '75+'
  else: 
    return ''

age_banding_udf = udf(bandingFunction, StringType() )

def addAgeBanding(df):
  df = df.withColumn('age_banding', age_banding_udf(df.age))
  return df.drop('age')

trainingSDF = addAgeBanding(trainingSDF)

trainingSDF.createOrReplaceTempView(temp_table_name)

Let's now visualize the distribution.

NOTE: as an alternative to Python-based plotting libraries like *seaborn* or *pyplot* we can also use Databricks' built-in visualizations. Click on the Visualization button below the results of this cell to select a **Bar** chart.

In [29]:
%sql
select age_banding, count(*) as Counts from trainingSDF group by age_banding order by age_banding

age_banding,Counts
18-25,1587
25-29,5084
30-34,7817
35-39,9573
40-44,11724
45-49,13828
50-54,13678
55-59,12851
60-64,12986
65-69,8844


### MonthlyIncome variable

In credit scoring, the income of the individual - besides the other debt that he is into - is of greater importance than other things when it comes to the final decision.

Let's see how the distribution of this variable looks.

In [31]:
fig, ax = plt.subplots()

ax = sns.boxplot(x="SeriousDlqin2yrs", y="MonthlyIncome", data = trainingSDF.toPandas())

display(fig)

Hmm, the chart isn't that useful, probably because we have some very large outliers (large *MonthlyIncome* values) skewing the plot. Let's try using a log scale for the y axis:

In [33]:
fig, ax = plt.subplots()

sns.set(style="whitegrid")
ax = sns.boxplot(x="SeriousDlqin2yrs", y="MonthlyIncome", data = trainingSDF.toPandas())
ax.set_yscale("log")

display(fig)

We can also display the quartile values of MonthlyIncome for each class, using Spark SQL.

In [35]:
%sql

SELECT SeriousDlqin2yrs, 
  percentile(MonthlyIncome,0.25) AS Q1, 
  percentile(MonthlyIncome,0.5) AS Q2_Median, 
  percentile(MonthlyIncome,0.75) AS Q3
FROM trainingSDF 
GROUP BY SeriousDlqin2yrs

SeriousDlqin2yrs,Q1,Q2_Median,Q3
1,2977.0,4500.0,6750.0
0,3475.0,5476.0,8333.0


That's better. One thing is certain - people which have gone through issues usually have a lower income. However, from the original summary statistics it also looked like the dataset contained really low values - like 5$ or less a month which is really odd.

For our reference, let's view the [Characteristics of Minimum Wage Workers in the US: 2010](https://www.bls.gov/cps/minwage2010.htm). In this article, it is stated that the prevailing Federal minimum wage was $7.25 per hour.

In this case, considering an individual would work on a full-time basis for 52 weeks straight in a year, that individual would earn **$7.25 X 40 hrs X 52 weeks** = **_$15,080_**.

This translates to approximately **_$1,256_** a month. For a part-time worker, this would mean a wage of **_$628_**. For an individual working only a quarter of the total time, that would mean a wage of only **_$314_**.

According to the [US Census Bureau, Current Population Survey 2016](https://en.wikipedia.org/wiki/Personal_income_in_the_United_States#cite_note-CPS_2015-2), 6.48% of people earned **_$2,500_** or less in a full year. This translates to only **_$208_** a month. Median personal income comes to about **_$31,099_** a year, which is about **_$2,592_** dollars a month.

Given all this information, let's do some more exploratory data analysis to see where this odd **MonthlyIncome** needs patching a bit.

Off the bat, there is weirdness in having NULL **MonthlyIncome** data, but being able to calculate **DebtRatio**.

In [38]:
%sql
select avg(DebtRatio), count(1) as Instances from trainingSDF where MonthlyIncome is null

avg(DebtRatio),Instances
1676.9326513213982,22287


It may be the case than whoever gathered this data have replaced **NULL** in this column to 1 to be able to calculate the **DebtRatio** using the data about the **TotalDebt** of the individual they had. This will be need to be treated this way:
- impute the **NULL** values with the median of the dataset
- recalculate the **DebtRatio** given we know that the **TotalDebt** is currently equal for those individuals to the value of the **DebtRatio**

A very low **MonthlyIncome** between $1 and $7 is again a bit suspicious (having worked under 1hr per month). Let's see a list of people with very small monthly incomes:

In [41]:
%sql
select MonthlyIncome, count(1) as Instances, avg(DebtRatio) from trainingSDF where MonthlyIncome between 1 and 100 group by MonthlyIncome order by 1

MonthlyIncome,Instances,avg(DebtRatio)
1,448,1009.6183035714286
2,3,206.33333333333334
4,2,49.5
5,2,114.6666666835
7,1,12.5
10,2,239.90909089999997
15,1,155.375
21,1,106.2727273
25,1,149.4230769
27,2,26.357142855


Given the number of records where **MonthlyIncome** is equal to 1 is suspiciously high, we are going to impute it like we do for the **NULL** values. However, for the other values, there isn't just too much wrong data to draw any conclusions. If we extend the window up to 208:

In [43]:
%sql
select count(1) as Instances from trainingSDF where MonthlyIncome between 2 and 208

Instances
103


100-odd rows is a low percentage of samples from the whole dataset, so for now we will be keeping these as they are.

That's quite a lot of information, so let's wrap up what we are going to do:

For the specifics of this lab, we are going to consider that:
- observations with a MonthlyIncome of 1 will be processed to get the median MonthlyIncome
- observations with a MonthlyIncome of null will be processed to get the median MonthlyIncome

Given the **DebtRatio** has been computed as the overall **Debt** divided by the **MonthlyIncome**, we are going to regenerate the initial debt first so we can use it later to recompute the **DebtRatio** based on the then cleaned **MonthlyIncome**.

First, we save the initial **Debt** so we are able to recompute the updated DebtRatio afterwards.

In [47]:
from pyspark.sql import functions as F

def addInitialDebtColumn(df):
  df = df.withColumn(
                  'initialDebt',
                  F.when(
                      (((F.col('MonthlyIncome') >= 0) & (F.col('MonthlyIncome') <= 1)) | (F.col('MonthlyIncome').isNull())),
                      F.col('DebtRatio')
                  ).otherwise(
                      F.col('MonthlyIncome') * F.col('DebtRatio')
                  )
              )
  
  return df
  
trainingSDF = addInitialDebtColumn(trainingSDF)

In [48]:
display(trainingSDF)

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,age_banding,initialDebt
1,1,0.766126609,2,0.802982129,9120,13,0,6,0,2,45-49,7323.19701648
2,0,0.957151019,0,0.121876201,2600,4,0,0,0,1,40-44,316.8781226
3,0,0.65818014,1,0.085113375,3042,2,1,0,0,0,35-39,258.91488675
4,0,0.233809776,0,0.036049682,3300,5,0,0,0,0,30-34,118.9639506
6,0,0.213178682,0,0.375606969,3500,3,0,1,0,1,70-74,1314.6243915
7,0,0.305682465,0,5710.0,null,8,0,3,0,0,55-59,5710.0
8,0,0.754463648,0,0.209940017,3500,8,0,0,0,0,35-39,734.7900595
9,0,0.116950644,0,46.0,null,2,0,0,0,null,25-29,46.0
10,0,0.189169052,0,0.606290901,23684,9,0,4,0,2,55-59,14359.393699284
12,0,0.01879812,0,0.53152876,6501,7,0,2,0,2,50-54,3455.46846876


After the initial **Debt** has been saved, we are good to start imputing the **MonthlyIncome** column. 
If the actual value is <= $7 or missing, we manually impute using the **numpy**-calculated median.

In [50]:
def imputeMonthlyIncome(df, incomeMedian):
  # Apply income median if the MonthlyIncome is <=7, or null
  
  df = df.withColumn('MonthlyIncome',
                                       F.when(
                                           (((F.col('MonthlyIncome') >= 0) & (F.col('MonthlyIncome') <= 7)) | (F.col('MonthlyIncome').isNull())),
                                           incomeMedian
                                       ).otherwise(
                                           F.col('MonthlyIncome')
                                       )
                )
  
  return df

trainingIncomeMedian = np.median(trainingSDF.select('MonthlyIncome').dropna().collect())
trainingSDF = imputeMonthlyIncome(trainingSDF, trainingIncomeMedian)

Now that the **MonthlyIncome** variable has been imputed, let's recalculate a more correct **DebtRatio** based on the initial **Debt** we have saved previously.

In [52]:
def recalculateDebtRatio(df):
  df = df.withColumn(
                    'DebtRatio',
                    df.initialDebt/df.MonthlyIncome
                )
  
  return df

trainingSDF = recalculateDebtRatio(trainingSDF)

trainingSDF.createOrReplaceTempView(temp_table_name)

Let's see how many values in this column are actually exceeding the threshold of **1** now.

In [54]:
%sql
select count(1) from trainingSDF where DebtRatio > 1

count(1)
4723


Let's see how it looks from a distribution point of view.

In [56]:
fig, ax = plt.subplots()

ax = sns.boxplot(x="DebtRatio", data = trainingSDF.toPandas())
ax.set_xscale("log")

display(fig)

It seems this values are going up into the hundreds. Individuals may exceed a **DebtRatio** of 1 whenever they are lending more than they are earning (and some people in difficult scenarios tend to do that).

Let's default the higher values to a threshold of **1.5**.

In [58]:
def defaultDebtRatioToThreshold(df):
  df = df.withColumn('DebtRatio',
                                       F.when(
                                           (F.col('DebtRatio') > 1.5),
                                           1.5
                                       ).otherwise(
                                           F.col('DebtRatio')
                                       )
                )
  
  return df

trainingSDF = defaultDebtRatioToThreshold(trainingSDF)

trainingSDF.createOrReplaceTempView(temp_table_name)

### RevolvingUtilizationOfUnsecuredLines variable
Let's understand how many values exceed 1 for this column and default them to this max value.

In [60]:
%sql
select count(1) from trainingSDF where RevolvingUtilizationOfUnsecuredLines > 1

count(1)
2485


Some records have a **RevolvingUtilizationOfUnsecuredLines** value higher than 1. Given the total balance on credit cards and personal lines of credit is divided to the sum of credit limits, this should not exceed 1.

Let's view the distribution of it and then default the weird records to this threshold.

In [62]:
fig, ax = plt.subplots()

ax = sns.boxplot(x="RevolvingUtilizationOfUnsecuredLines", data = trainingSDF.toPandas())
ax.set_xscale("log")


display(fig)

In [63]:
def defaultRevolvingUtilizationToThreshold(df):
  df = df.withColumn('RevolvingUtilizationOfUnsecuredLines',
                                       F.when(
                                           (F.col('RevolvingUtilizationOfUnsecuredLines') > 1),
                                           1
                                       ).otherwise(
                                           F.col('RevolvingUtilizationOfUnsecuredLines')
                                       )
                )
  
  return df

trainingSDF = defaultRevolvingUtilizationToThreshold(trainingSDF)

trainingSDF.createOrReplaceTempView(temp_table_name)

### NumberOfDependents variable

Let's understand how many missing values this column has.

In [65]:
%sql
select count(1) from trainingSDF where NumberOfDependents is null

count(1)
2937


About 3000 missing values out of the total number of rows is not bad at all.

Let's see how the distribution of this variable looks. We will understand the mode from it and will be able to impute using it.

In [67]:
# spark.sql does not have any histogram method, however the RDD api does
dependents_histogram = trainingSDF.select('NumberOfDependents').rdd.flatMap(lambda x: x).histogram(10)

fig, ax = plt.subplots()

# the computed histogram needs to be loaded in a pandas dataframe so we will be able to plot it using sns
dependents_histogram_df = pd.DataFrame(
    list(zip(*dependents_histogram)), 
    columns=['bin', 'count']
)

ax = sns.barplot(x = "bin", y = "count", data = dependents_histogram_df)

display(fig)

We can tell from the barplot above that the mode (most frequent value) of this column is 0. Let's impute the missing values with it.

In [69]:
def imputeNumberOfDependents(df):
  df = df.withColumn('NumberOfDependents',
                                       F.when(
                                           (F.col('NumberOfDependents').isNull()),
                                           0
                                       ).otherwise(
                                           F.col('NumberOfDependents')
                                       )
                )

  return df

trainingSDF = imputeNumberOfDependents(trainingSDF)
  
trainingSDF.createOrReplaceTempView(temp_table_name)

In [70]:
# Check of the summary statistics of the features now
display(trainingSDF.describe())

summary,Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,age_banding,initialDebt
count,112610,112610,112610,112610,112610,112610,112610,112610,112610,112610,112610,112610,112610
mean,74982.02763520114,0.06655714412574372,0.31930370332861197,0.4257348370482195,0.3367918340611968,6479.482639197229,8.450031080721073,0.27232039783322975,1.0170411153538763,0.24635467542846995,0.7347748867773732,null,2042.5903747831078
stddev,43267.1824110714,0.2492545734778176,0.34996321667984515,4.263688445126266,0.3056866892439673,10996.874198706579,5.1483988188414065,4.240935535626126,1.1218679345702816,4.226870854098003,1.1009547472610703,null,4223.602442158943
min,1,0,0.0,0,0.0,10.0,0,0,0,0,0,18-25,0.0
max,149999,1,1.0,98,1.5,1794060.0,57,98,29,98,10,75+,478450.55915999995


## Building our first model

For our first attempt at building a model we will use a relatively simple algorithm, Decision Trees.

![Decision Tree Example](https://miro.medium.com/max/792/1*XMId5sJqPtm8-RIwVVz2tg.png)

[Click here](https://www.youtube.com/watch?v=7VeUPuFGJHk) for a straightforward video explanation of how Decision Trees work, and how we can build one using Gini Impurity.

In [72]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler

# Index categorical features
categorical_indexer = StringIndexer(inputCol="age_banding", outputCol="age_banding_indexed")

# assemble all features into a features vector
feature_assembler = VectorAssembler(
    inputCols=[
   'RevolvingUtilizationOfUnsecuredLines',
   'NumberOfTime30-59DaysPastDueNotWorse',
   'NumberOfOpenCreditLinesAndLoans',
   'NumberOfTimes90DaysLate',
   'NumberRealEstateLoansOrLines',
   'NumberOfTime60-89DaysPastDueNotWorse',
   'NumberOfDependents',
   'age_banding_indexed',
   'initialDebt',
   'DebtRatio',
   'MonthlyIncome'],
    outputCol="features")


# Train a DecisionTree model.
decision_tree_classifier = DecisionTreeClassifier(labelCol="SeriousDlqin2yrs", featuresCol="features",
                            impurity="gini", maxDepth=5, seed=1)

# Chain assembler and model in a Pipeline
dtc_pipeline = Pipeline(stages=[categorical_indexer, feature_assembler, decision_tree_classifier])

# Train model. 
dtc_model = dtc_pipeline.fit(trainingSDF)

print(dtc_model.stages[2])

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_86c2ce5db91b) of depth 5 with 31 nodes

In [73]:
#let's get a text-based representation of the tree
print(dtc_model.stages[2].toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_86c2ce5db91b) of depth 5 with 31 nodes
 If (feature 3 <= 0.5)
 If (feature 0 <= 0.49489401050000004)
 Predict: 0.0
 Else (feature 0 > 0.49489401050000004)
 If (feature 5 <= 0.5)
 Predict: 0.0
 Else (feature 5 > 0.5)
 If (feature 0 <= 0.9999999500000001)
 Predict: 0.0
 Else (feature 0 > 0.9999999500000001)
 If (feature 2 <= 6.5)
 Predict: 0.0
 Else (feature 2 > 6.5)
 Predict: 1.0
 Else (feature 3 > 0.5)
 If (feature 3 <= 1.5)
 If (feature 0 <= 0.49489401050000004)
 If (feature 5 <= 0.5)
 Predict: 0.0
 Else (feature 5 > 0.5)
 If (feature 8 <= 3976.8011699999997)
 Predict: 0.0
 Else (feature 8 > 3976.8011699999997)
 Predict: 1.0
 Else (feature 0 > 0.49489401050000004)
 If (feature 5 <= 0.5)
 Predict: 0.0
 Else (feature 5 > 0.5)
 If (feature 8 <= 3286.8396755000003)
 Predict: 0.0
 Else (feature 8 > 3286.8396755000003)
 Predict: 1.0
 Else (feature 3 > 1.5)
 If (feature 1 <= 0.5)
 If (feature 5 <= 1.5)
 Predict: 0.0
 Else (feature 5 > 1.5)
 Predict: 1.0
 Else (feature 1 > 0.5)
 If (feature 9 <= 0.500179443)
 If (feature 8 <= 2.9998567740000004)
 Predict: 0.0
 Else (feature 8 > 2.9998567740000004)
 Predict: 1.0
 Else (feature 9 > 0.500179443)
 Predict: 1.0

# Testing the model

We will now test the model by predicting on the test data. Once we obtain predictions, we use the predictions and the ground truth values from the test dataset to compute binary classification evaluation metrics.

Before we can use the model, we need to apply to the test data the same transformations we did in the preprocessing stage.

Notice that se are using statistical values like `trainingMedianAge` which were computed on the training data set. It's good practice to treat the test dataset as completely new information, and not use it in any way except actually testing our ML pipeline.

In [75]:
testingSDF = imputeAgeWithMedian(testingSDF, trainingMedianAge)
testingSDF = addAgeBanding(testingSDF)
testingSDF = addInitialDebtColumn(testingSDF)
testingSDF = imputeMonthlyIncome(testingSDF, trainingIncomeMedian)
testingSDF = recalculateDebtRatio(testingSDF)
testingSDF = defaultDebtRatioToThreshold(testingSDF)
testingSDF = defaultRevolvingUtilizationToThreshold(testingSDF)
testingSDF = imputeNumberOfDependents(testingSDF)

# Make the dataframe available in the SQL context
test_temp_table_name = "testingSDF"

# Make the dataframe available in the SQL context
testingSDF.createOrReplaceTempView(test_temp_table_name)

display(testingSDF)

Idx,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,age_banding,initialDebt
5,0,0.9072394,1,0.024925695,63588.0,7,0,1,0,0,45-49,1584.97509366
11,0,0.644225962,0,0.30947621,2500.0,5,0,0,0,0,30-34,773.690525
17,0,0.061086118,0,0.3811111111111111,5400.0,10,0,2,0,0,75+,2058.0
19,0,0.221812771,0,0.527887839,3280.0,7,0,1,0,2,40-44,1731.47211192
21,0,0.200923382,0,0.43004633800000003,12300.0,10,0,2,0,0,40-44,5289.5699574
25,0,0.04656027,0,0.24162184499999997,2416.0,9,0,1,0,0,55-59,583.75837752
33,0,0.083418009,0,0.18092592592592593,5400.0,6,0,1,0,0,60-64,977.0
37,0,0.881836362,3,0.567858035,4000.0,9,0,1,0,1,50-54,2271.43214
39,0,0.363636364,0,0.00999001,1000.0,1,0,0,0,0,25-29,9.99001
41,0,0.71998531,1,0.539554464,5700.0,14,0,1,0,0,40-44,3075.4604447999996


In [76]:
# Make predictions.
dtc_predictions = dtc_model.transform(testingSDF)

# Select example rows to display.
display(dtc_predictions.select("probability", "prediction", "SeriousDlqin2yrs"))


probability,prediction,SeriousDlqin2yrs
"List(1, 2, List(), List(0.901157830591103, 0.09884216940889701))",0.0,0
"List(1, 2, List(), List(0.901157830591103, 0.09884216940889701))",0.0,0
"List(1, 2, List(), List(0.9777245418856376, 0.022275458114362403))",0.0,0
"List(1, 2, List(), List(0.9777245418856376, 0.022275458114362403))",0.0,0
"List(1, 2, List(), List(0.9777245418856376, 0.022275458114362403))",0.0,0
"List(1, 2, List(), List(0.9777245418856376, 0.022275458114362403))",0.0,0
"List(1, 2, List(), List(0.9777245418856376, 0.022275458114362403))",0.0,0
"List(1, 2, List(), List(0.901157830591103, 0.09884216940889701))",0.0,0
"List(1, 2, List(), List(0.9777245418856376, 0.022275458114362403))",0.0,0
"List(1, 2, List(), List(0.901157830591103, 0.09884216940889701))",0.0,0


First, we're going to calculate and display the [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for our binary classifier.

In [78]:
# display the confusion matrix
from sklearn.metrics import confusion_matrix

def plotConfusionMatrix(confusion_matrix):
  fig, ax = plt.subplots()
  plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Wistia)
  classNames = ['Negative','Positive']
  ax.set_title(f'Confusion Matrix')
  ax.set_ylabel('True label')
  ax.set_xlabel('Predicted label')
  tick_marks = np.arange(len(classNames))
  ax.set_xticks(tick_marks)
  ax.set_yticks(tick_marks)
  s = [['TN','FP'], ['FN', 'TP']]
  for i in range(2):
      for j in range(2):
          ax.text(j,i, str(s[i][j])+" = "+str(confusion_matrix[i][j]))
  display(fig)
  
  
dtc_confusion_matrix = confusion_matrix(dtc_predictions.select("SeriousDlqin2yrs").collect(), dtc_predictions.select("prediction").collect())
plotConfusionMatrix(dtc_confusion_matrix)

###Precision and Recall

![Precision and Recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png)

In [80]:
tn, fp, fn, tp = dtc_confusion_matrix.ravel()
print(f"Accuracy = (TP + TN) / (TP + FP + TN + FN) = {(tp+tn)/(tp+fp+tn+fn)}")
print(f"Precision = TP / (TP + FP) = {tp/(tp+fp)}")
print(f"Recall = TP / (TP + FN) = {tp/(tp+fn)}")

Accuracy = (TP + TN) / (TP + FP + TN + FN) = 0.9348221449585451
Precision = TP / (TP + FP) = 0.5863970588235294
Recall = TP / (TP + FN) = 0.126037139470565

### Sensitivity and Specificity, and the ROC Curve

![Sensitivity and specificity](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/Sensitivity_and_specificity.svg/350px-Sensitivity_and_specificity.svg.png)

In [82]:
# plot the ROC curve
from sklearn.metrics import roc_curve, auc

def plotROCCurve(predictions, show_thresholds=False):
  results = predictions.select(['probability', 'SeriousDlqin2yrs']).collect()
  y_score = [float(i[0][1]) for i in results]
  y_true = [float(i[1]) for i in results]

  fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label = 1)
  roc_auc = auc(fpr, tpr)

  fig, ax = plt.subplots()
  ax.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
  ax.plot([0, 1], [0, 1], 'k--')
  if show_thresholds:
      tr_idx = np.arange(385, len(thresholds), 700)
      for i in tr_idx:
        ax.plot(fpr[i], tpr[i], "xr")
        ax.annotate(xy=(fpr[i], tpr[i]), s="%0.3f" % thresholds[i])
  ax.set_xlim([0.0, 1.0])
  ax.set_ylim([0.0, 1.0])
  ax.set_xlabel('False Positive Rate (1 - Specificity)')
  ax.set_ylabel('True Positive Rate (Sensitivity)')
  ax.set_title('Receiver operating characteristic')
  ax.legend(loc="lower right")
  display(fig)

plotROCCurve(dtc_predictions)

## Gradient Boosted Trees

Now we're going to try an ensemble model: [Gradient Boosted Trees](https://en.wikipedia.org/wiki/Gradient_boosting).

![GBT](http://uc-r.github.io/public/images/analytics/gbm/boosted-trees-process.png)

[Click here](https://www.youtube.com/watch?v=3CC4N4z3GJc) for a nice visual explanation of how Gradient Boosting works.

In [84]:
from pyspark.ml.classification import GBTClassifier

# scale features 
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

# Train a Gradient-boosted tree classifier model.
gbt_classifier = GBTClassifier(labelCol="SeriousDlqin2yrs", featuresCol="features",
                            maxIter=35, seed=1)

# Chain assembler and model in a Pipeline
gbt_pipeline = Pipeline(stages=[categorical_indexer, feature_assembler, scaler, gbt_classifier])

# Train model. 
gbt_model = gbt_pipeline.fit(trainingSDF)

print(gbt_model.stages[3])

GBTClassificationModel (uid=GBTClassifier_e6f7c02404b1) with 35 trees

In [85]:
print(gbt_model.stages[3].toDebugString)

GBTClassificationModel (uid=GBTClassifier_e6f7c02404b1) with 35 trees
 Tree 0 (weight 1.0):
 If (feature 3 <= 0.5)
 If (feature 0 <= 0.49489401050000004)
 If (feature 1 <= 0.5)
 If (feature 5 <= 0.5)
 If (feature 0 <= 0.2756211325)
 Predict: -0.9768772665269322
 Else (feature 0 > 0.2756211325)
 Predict: -0.9382891863666571
 Else (feature 5 > 0.5)
 If (feature 5 <= 1.5)
 Predict: -0.8216704288939052
 Else (feature 5 > 1.5)
 Predict: -0.4794520547945205
 Else (feature 1 > 0.5)
 If (feature 5 <= 0.5)
 If (feature 1 <= 1.5)
 Predict: -0.8884178395794472
 Else (feature 1 > 1.5)
 Predict: -0.7596948506039415
 Else (feature 5 > 0.5)
 If (feature 10 <= 9166.5)
 Predict: -0.5319148936170213
 Else (feature 10 > 9166.5)
 Predict: -0.8202247191011236
 Else (feature 0 > 0.49489401050000004)
 If (feature 5 <= 0.5)
 If (feature 1 <= 0.5)
 If (feature 7 in {1.0,2.0,3.0,6.0,7.0,9.0})
 Predict: -0.8975295119873433
 Else (feature 7 not in {1.0,2.0,3.0,6.0,7.0,9.0})
 Predict: -0.8220736994219653
 Else (feature 1 > 0.5)
 If (feature 0 <= 0.9999999500000001)
 Predict: -0.6549413735343383
 Else (feature 0 > 0.9999999500000001)
 Predict: -0.26181818181818184
 Else (feature 5 > 0.5)
 If (feature 0 <= 0.9999999500000001)
 If (feature 5 <= 1.5)
 Predict: -0.44719101123595506
 Else (feature 5 > 1.5)
 Predict: -0.17117117117117117
 Else (feature 0 > 0.9999999500000001)
 If (feature 2 <= 6.5)
 Predict: -0.18604651162790697
 Else (feature 2 > 6.5)
 Predict: 0.35384615384615387
 Else (feature 3 > 0.5)
 If (feature 3 <= 1.5)
 If (feature 0 <= 0.49489401050000004)
 If (feature 5 <= 0.5)
 If (feature 0 <= 0.23244911699999998)
 Predict: -0.7373737373737373
 Else (feature 0 > 0.23244911699999998)
 Predict: -0.5240963855421686
 Else (feature 5 > 0.5)
 If (feature 8 <= 3976.8011699999997)
 Predict: -0.2946859903381642
 Else (feature 8 > 3976.8011699999997)
 Predict: 0.36363636363636365
 Else (feature 0 > 0.49489401050000004)
 If (feature 5 <= 0.5)
 If (feature 1 <= 1.5)
 Predict: -0.35301668806161746
 Else (feature 1 > 1.5)
 Predict: -0.07065217391304347
 Else (feature 5 > 0.5)
 If (feature 8 <= 3286.8396755000003)
 Predict: -0.01037037037037037
 Else (feature 8 > 3286.8396755000003)
 Predict: 0.4339622641509434
 Else (feature 3 > 1.5)
 If (feature 1 <= 0.5)
 If (feature 5 <= 1.5)
 If (feature 0 <= 0.9343107214999999)
 Predict: -0.28825622775800713
 Else (feature 0 > 0.9343107214999999)
 Predict: -0.018115942028985508
 Else (feature 5 > 1.5)
 If (feature 9 <= 0.464765122)
 Predict: 0.12903225806451613
 Else (feature 9 > 0.464765122)
 Predict: 0.8461538461538461
 Else (feature 1 > 0.5)
 If (feature 9 <= 0.500179443)
 If (feature 7 in {1.0,6.0,11.0})
 Predict: -0.05485232067510549
 Else (feature 7 not in {1.0,6.0,11.0})
 Predict: 0.18536009445100354
 Else (feature 9 > 0.500179443)
 If (feature 0 <= 0.566460627)
 Predict: 0.08333333333333333
 Else (feature 0 > 0.566460627)
 Predict: 0.475177304964539
 Tree 1 (weight 0.1):
 If (feature 0 <= 0.7504790425000001)
 If (feature 1 <= 1.5)
 If (feature 3 <= 0.5)
 If (feature 9 <= 0.643730836)
 If (feature 0 <= 0.105040213)
 Predict: -0.4604245229723371
 Else (feature 0 > 0.105040213)
 Predict: -0.41822515031382607
 Else (feature 9 > 0.643730836)
 If (feature 4 <= 3.5)
 Predict: -0.3635261023908695
 Else (feature 4 > 3.5)
 Predict: -0.14718395683293167
 Else (feature 3 > 0.5)
 If (feature 7 in {2.0,3.0,4.0,5.0,6.0,7.0})
 If (feature 3 <= 5.5)
 Predict: -0.32015705484298346
 Else (feature 3 > 5.5)
 Predict: 1.1961053112933597
 Else (feature 7 not in {2.0,3.0,4.0,5.0,6.0,7.0})
 If (feature 5 <= 2.5)
 Predict: -0.03765618669764745
 Else (feature 5 > 2.5)
 Predict: 1.1942622726144838
 Else (feature 1 > 1.5)
 If (feature 1 <= 3.5)
 If (feature 7 in {1.0,2.0,3.0,4.0,6.0,7.0,9.0})
 If (feature 1 <= 2.5)
 Predict: -0.3048671937901954
 Else (feature 1 > 2.5)
 Predict: -0.09819771962083469
 Else (feature 7 not in {1.0,2.0,3.0,4.0,6.0,7.0,9.0})
 If (feature 7 in {0.0,5.0,8.0,10.0})
 Predict: -0.04272753619963082
 Else (feature 7 not in {

In [86]:
# Make predictions.
gbt_predictions = gbt_model.transform(testingSDF)

# Select example rows to display.
display(gbt_predictions.select("probability", "prediction", "SeriousDlqin2yrs"))

probability,prediction,SeriousDlqin2yrs
"List(1, 2, List(), List(0.8762975483918157, 0.12370245160818427))",0.0,0
"List(1, 2, List(), List(0.9348171464956012, 0.06518285350439879))",0.0,0
"List(1, 2, List(), List(0.9656553295885913, 0.03434467041140865))",0.0,0
"List(1, 2, List(), List(0.962966307908138, 0.037033692091862025))",0.0,0
"List(1, 2, List(), List(0.9617795564575178, 0.03822044354248222))",0.0,0
"List(1, 2, List(), List(0.9661702065264454, 0.03382979347355464))",0.0,0
"List(1, 2, List(), List(0.9667393743527671, 0.033260625647232867))",0.0,0
"List(1, 2, List(), List(0.7435918832721903, 0.2564081167278097))",0.0,0
"List(1, 2, List(), List(0.9469867365753477, 0.05301326342465229))",0.0,0
"List(1, 2, List(), List(0.8321999150513707, 0.1678000849486293))",0.0,0


In [87]:
gbt_confusion_matrix = confusion_matrix(gbt_predictions.select("SeriousDlqin2yrs").collect(), gbt_predictions.select("prediction").collect())
plotConfusionMatrix(gbt_confusion_matrix)

In [88]:
tn, fp, fn, tp = gbt_confusion_matrix.ravel()
print(f"Precision = TP / (TP + FP) = {tp/(tp+fp)}")
print(f"Recall = TP / (TP + FN) = {tp/(tp+fn)}")
print(f"Sensitivity = TP / (TP + FN) = {tp/(tp+fn)}")
print(f"Specificity = TN / (TN + FP) = {tn/(tn+fp)}")

Precision = TP / (TP + FP) = 0.5830258302583026
Recall = TP / (TP + FN) = 0.18727775582773606
Sensitivity = TP / (TP + FN) = 0.18727775582773606
Specificity = TN / (TN + FP) = 0.9902751082934106

In [89]:
plotROCCurve(gbt_predictions)

### Selecting a better threshold for class separation

In [91]:
plotROCCurve(gbt_predictions, show_thresholds = True)

In [92]:
# select a different threshold for class separation, make predictions based on that threshold, and recalculate Precision, Recall, Sensitivity and Specificity.
from pyspark.sql.types import FloatType

get_positive_probability=udf(lambda v:float(v[1]),FloatType())


selected_threshold = 0.11
pred_colname = f'prediction-threshold'
gbt_predictions_threshold = gbt_predictions.withColumn(pred_colname, 
                                       F.when(get_positive_probability('probability') <= selected_threshold,0)
                                                       .otherwise(1))
                                                       
display(gbt_predictions_threshold.select("probability", "prediction", pred_colname, "SeriousDlqin2yrs"))                                                

probability,prediction,prediction-threshold,SeriousDlqin2yrs
"List(1, 2, List(), List(0.8762975483918157, 0.12370245160818427))",0.0,1,0
"List(1, 2, List(), List(0.9348171464956012, 0.06518285350439879))",0.0,0,0
"List(1, 2, List(), List(0.9656553295885913, 0.03434467041140865))",0.0,0,0
"List(1, 2, List(), List(0.962966307908138, 0.037033692091862025))",0.0,0,0
"List(1, 2, List(), List(0.9617795564575178, 0.03822044354248222))",0.0,0,0
"List(1, 2, List(), List(0.9661702065264454, 0.03382979347355464))",0.0,0,0
"List(1, 2, List(), List(0.9667393743527671, 0.033260625647232867))",0.0,0,0
"List(1, 2, List(), List(0.7435918832721903, 0.2564081167278097))",0.0,1,0
"List(1, 2, List(), List(0.9469867365753477, 0.05301326342465229))",0.0,0,0
"List(1, 2, List(), List(0.8321999150513707, 0.1678000849486293))",0.0,1,0


In [93]:
gbt_threshold_confusion_matrix = confusion_matrix(gbt_predictions_threshold.select("SeriousDlqin2yrs").collect(), gbt_predictions_threshold.select("prediction-threshold").collect())
plotConfusionMatrix(gbt_threshold_confusion_matrix)

In [94]:
tn, fp, fn, tp = gbt_threshold_confusion_matrix.ravel()

print(f"Precision = TP / (TP + FP) = {tp/(tp+fp)}")
print(f"Recall = TP / (TP + FN) = {tp/(tp+fn)}")

print(f"Sensitivity = TP / (TP + FN) = {tp/(tp+fn)}")
print(f"Specificity = TN / (TN + FP) = {tn/(tn+fp)}")

Precision = TP / (TP + FP) = 0.2831873604671132
Recall = TP / (TP + FN) = 0.651521137890162
Sensitivity = TP / (TP + FN) = 0.651521137890162
Specificity = TN / (TN + FP) = 0.8802604779253564

## Hyperparameter Tuning

Until now we've built a Gradient Boosted classfier with just the default parameters (except `maxIter` which we set to 35).
It would be useful to optimize the parameters for the algorithm (also called hyperparameters).

[Hyperparameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization) or tuning is the problem of choosing a set of optimal hyperparameters for a learning algorithm. A hyperparameter is a parameter whose value is used to control the learning process. By contrast, the values of other parameters (typically node weights) are learned.

We will combine Hyperparameter Tuning with [Cross-Validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) on the training dataset, so we are able to even out the noise in the training data. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice. 

![HT and CV](https://cambridgecoding.files.wordpress.com/2016/03/gridsearch_cv.png)

In [96]:
print(gbt_classifier.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: all)
featuresCol: features column name. (default: features, current: features)
labelCol: label column name. (default: label, current: SeriousDlqin2yrs)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxIter: max number of iterations (>= 0). (default: 20, current: 35)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 8078277157982239434, current: 1)
stepSize: Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator. (default: 0.1)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)

In [97]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator


paramGrid = (ParamGridBuilder()
             .addGrid(gbt_classifier.maxDepth, [5, 8])
             .addGrid(gbt_classifier.maxIter, [25, 40])
             .addGrid(gbt_classifier.stepSize, [0.1, 0.2])
             .build())


evaluator = BinaryClassificationEvaluator(
  rawPredictionCol="prediction", labelCol="SeriousDlqin2yrs", metricName="areaUnderROC")

cv = CrossValidator(estimator=gbt_pipeline, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

# Train model. 
gbt_models_cv = cv.fit(trainingSDF)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [98]:
best_model = gbt_models_cv.bestModel.stages[3]
print(best_model.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: all)
featuresCol: features column name (default: features, current: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (undefined)
labelCol: label column name (default: label, current: SeriousDlqin2yrs)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
maxBins: Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5, current: 8)
maxIter: maximum number of iterations (>= 0) (default: 20, current: 40)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1. (default: 1)
predictionCol: prediction column name (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities (undefined)
rawPredictionCol: raw prediction (a.k.a. confidence) column name (undefined)
seed: random seed (default: 8078277157982239434, current: 1)
stepSize: Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator. (default: 0.1, current: 0.2)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0 excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold (undefined)
validationIndicatorCol: name of the column that indicates whether each row is for training or for validation. False indicates training; true indicates validation. (undefined)
validationTol: Threshold for stopping early when fit with validation is used.If the error rate on the validation input changes by less than the validationTol,then learning will stop early (before `maxIter`).This parameter is ignored when fit without validation is used. (undefined)

In [99]:
# Make predictions.
gbt_cv_predictions = gbt_models_cv.transform(testingSDF)

# Select example rows to display.
display(gbt_cv_predictions.select("probability", "prediction", "SeriousDlqin2yrs"))

In [100]:
plotROCCurve(gbt_cv_predictions, show_thresholds = True)

In [101]:
selected_threshold = 0.11
pred_colname = f'prediction-threshold'
gbt_cv_predictions_threshold = gbt_cv_predictions.withColumn(pred_colname, 
                                       F.when(get_positive_probability('probability') < selected_threshold,0)
                                                       .otherwise(1))
                                                       
display(gbt_cv_predictions_threshold.select("probability", "prediction", pred_colname, "SeriousDlqin2yrs"))  

In [102]:
gbt_cv_threshold_confusion_matrix = confusion_matrix(gbt_cv_predictions_threshold.select("SeriousDlqin2yrs").collect(), gbt_cv_predictions_threshold.select("prediction-threshold").collect())
plotConfusionMatrix(gbt_cv_threshold_confusion_matrix)

In [103]:
tn, fp, fn, tp = gbt_cv_threshold_confusion_matrix.ravel()

print(f"Precision = TP / (TP + FP) = {tp/(tp+fp)}")
print(f"Recall = TP / (TP + FN) = {tp/(tp+fn)}")

print(f"Sensitivity = TP / (TP + FN) = {tp/(tp+fn)}")
print(f"Specificity = TN / (TN + FP) = {tn/(tn+fp)}")

## Using Automated ML from Azure ML Service

We are now going to use the [AutoML feature](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml) from the Azure Machine Learning Service SDK.

Automated machine learning, also referred to as automated ML, is the process of automating the time consuming, iterative tasks of machine learning model development. It allows data scientists, analysts, and developers to build ML models with high scale, efficiency, and productivity all while sustaining model quality. Automated ML is based on a breakthrough from our Microsoft Research division.

Traditional machine learning model development is resource-intensive, requiring significant domain knowledge and time to produce and compare dozens of models. Apply automated ML when you want Azure Machine Learning to train and tune a model for you using the target metric you specify. The service then iterates through ML algorithms paired with feature selections, where each iteration produces a model with a training score. The higher the score, the better the model is considered to "fit" your data.


We will provide the cleansed training data to Azure ML which will test multiple types of algorithms in order to maximize a certain evaluation criteria we define. As per the [initial challenge from kaggle](https://www.kaggle.com/c/GiveMeSomeCredit), the criteria of choice is AUC (Area Under Curve).

The validation during training is done by using cross validation in 5 folds.

After we are done, the best trained model will be evaluated against a separated dataset (the test dataset) in order to understand real _performance_.

### Training using AutoML

In order to get things going, we first initialize our Workspace...

In [105]:
subscription_id = "6787a35f-386b-4845-91d1-695f24e0924b" # the Azure subscription ID you are using
azureml_resource_group = "spark-ml-workshop-25" #you should be owner or contributor
azureml_workspace_name = "azureml-lab-25" #your Azure Machine Learning workspace name

import azureml.core

# Check core SDK version number - based on build number of preview/master.
print("Azure ML SDK version:", azureml.core.VERSION)

from azureml.core import Workspace

ws = Workspace(workspace_name = azureml_workspace_name,
               subscription_id = subscription_id,
               resource_group = azureml_resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

In [106]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

And then we make sure we have all the important libraries in place.

In [108]:
import logging
import os
import random
import time

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

We prepare the experiment properties which will be provided once we issue a training request.

In [110]:
# Get the last seven letters of the username which will be used to build up exp name
import re

regexStr = r'^([^@]+)@[^@]+$'
emailStr = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply("user")
matchobj = re.search(regexStr, emailStr)
if not matchobj is None:
    if len(matchobj.group(1)) > 10:
        notebook_username = matchobj.group(1)[-10:]
    else:
        notebook_username = matchobj.group(1)
        
    print(notebook_username)
else:
    print("Did not match")

In [111]:
# Choose a name for the experiment and specify the project folder.
experiment_base_name = 'automl-scoring-'
experiment_suffix_name = notebook_username.replace(".", "") + "-" + str(random.randint(1000, 9999))

experiment_name = experiment_base_name + experiment_suffix_name
project_folder = './globalainight_projects/automl-credit-scring'

print(experiment_name)

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

Enabling diagnostics to understand better what's going on.

In [113]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

We save the cleansed training data to CSV files in the DBFS and then we load it separately in two dataflows:
- **X_train**: contains **_only_** the training variables
- **Y_train**: contains **_only_** the result

In [115]:
# prepare the data for AutoML
import azureml.dataprep as dataprep

training_sdf = trainingSDF
training_sdf = training_sdf.drop("Idx", "initialDebt")

training_sdf \
.drop("SeriousDlqin2yrs") \
.toPandas() \
.to_csv("/dbfs/FileStore/tables/constant-scoring-training-vars.csv")

training_sdf \
.select("SeriousDlqin2yrs") \
.toPandas() \
.to_csv("/dbfs/FileStore/tables/constant-scoring-training-res.csv")

X_train = dataprep.read_csv(path = "/dbfs/FileStore/tables/constant-scoring-training-vars.csv", separator = ',')
X_train = X_train.drop_columns("Column1")

Y_train = dataprep.read_csv(path = "/dbfs/FileStore/tables/constant-scoring-training-res.csv", separator = ',')
Y_train = Y_train.drop_columns("Column1")

Checking to make sure we have data inside.

In [117]:
X_train.head(5)

In [118]:
Y_train.head(5)

AutoML in Azure may be configured by passing multiple properties through the [AutoML Config](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlconfig?view=azure-ml-py).

We are interested in the following:

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**spark_context**|Spark Context object. for Databricks, use spark_context=sc|
|**max_concurrent_iterations**|Maximum number of iterations to execute in parallel. This should be <= number of worker nodes in your Azure Databricks cluster.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [120]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 5,
                             iterations = 10,
                             n_cross_validations = 5,
                             max_concurrent_iterations = 1, 
                             verbosity = logging.INFO,
                             spark_context=sc, #databricks/spark related
                             X = X_train, 
                             y = Y_train,
                             path = project_folder,
                             preprocess = True,
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False)

We are now ready to submit a new run for our experiment.

In [122]:
local_run = experiment.submit(automl_config, show_output = True) # for higher runs please use show_output=False and use the below

And once the run is finished, we are able to retrieve the best model as per the metric we have configured.

In [124]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

In [125]:
print(fitted_model.steps)

**Portal URL for Monitoring Runs**

The following will provide a link to the web interface to explore individual run details and status.

In [127]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

### Evaluating the best model from AutoML

Now that we are done with training, we can move forward in evaluating how well this model will actually do on the test data.

In [129]:
automl_X_test = testingSDF.drop("Idx", "initialDebt","SeriousDlqin2yrs")
automl_Y_test = testingSDF.select("SeriousDlqin2yrs")

In [130]:
automl_predictions_pd = fitted_model.predict_proba(automl_X_test.toPandas())
tempdf = pd.concat([pd.DataFrame(automl_predictions_pd), automl_Y_test.toPandas()], axis=1)
automl_predictions = spark.createDataFrame(tempdf)
display(automl_predictions)

In [131]:
def plotROCCurve2(predictions, show_thresholds=False):
  results = predictions.select(['1', 'SeriousDlqin2yrs']).collect()
  y_score = [float(i[0]) for i in results]
  y_true = [float(i[1]) for i in results]

  fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label = 1)
  roc_auc = auc(fpr, tpr)

  fig, ax = plt.subplots()
  ax.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
  ax.plot([0, 1], [0, 1], 'k--')
  if show_thresholds:
      tr_idx = np.arange(385, len(thresholds), 700)
      for i in tr_idx:
        ax.plot(fpr[i], tpr[i], "xr")
        ax.annotate(xy=(fpr[i], tpr[i]), s="%0.3f" % thresholds[i])
  ax.set_xlim([0.0, 1.0])
  ax.set_ylim([0.0, 1.0])
  ax.set_xlabel('False Positive Rate (1 - Specificity)')
  ax.set_ylabel('True Positive Rate (Sensitivity)')
  ax.set_title('Receiver operating characteristic')
  ax.legend(loc="lower right")
  display(fig)

plotROCCurve2(automl_predictions, show_thresholds = True)

In [132]:
selected_threshold = 0.12
pred_colname = f'prediction-threshold'
automl_predictions_threshold = automl_predictions_SDF.withColumn(pred_colname, 
                                       F.when(F.col('1') < selected_threshold, 0)
                                                       .otherwise(1))
display(automl_predictions_threshold)

In [133]:
automl_threshold_confusion_matrix = confusion_matrix(automl_predictions_threshold.select("SeriousDlqin2yrs").collect(), automl_predictions_threshold.select("prediction-threshold").collect())
plotConfusionMatrix(automl_threshold_confusion_matrix)

In [134]:
tn, fp, fn, tp = automl_threshold_confusion_matrix.ravel()

print(f"Precision = TP / (TP + FP) = {tp/(tp+fp)}")
print(f"Recall = TP / (TP + FN) = {tp/(tp+fn)}")

print(f"Sensitivity = TP / (TP + FN) = {tp/(tp+fn)}")
print(f"Specificity = TN / (TN + FP) = {tn/(tn+fp)}")